In [ ]:
# Import Necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import TensorFlow for building the neural network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Import scikit-learn utilities for evaluation
from sklearn.metrics import classification_report, confusion_matrix

# Load and Prepare the Data
try:
    train_df = pd.read_csv('data/fashion-mnist_train.csv')
    test_df = pd.read_csv('data/fashion-mnist_test.csv')
    print("Successfully loaded data from local CSV files.")
except FileNotFoundError:
    print("Error: CSV files not found.")
    exit()

# Data Preprocessing
y_train = train_df['label'].values
X_train = train_df.drop('label', axis=1).values

y_test = test_df['label'].values
X_test = test_df.drop('label', axis=1).values

# Classify Labels by making where in the list it is equal to the number.
Labels = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

# The images are 28x28 pixels so they need to be reshaped to fit that.
X_train = X_train.reshape(-1, 28, 28)
X_test = X_test.reshape(-1, 28, 28)

# Since each pixel is just darkness between 1 to 255 it can be normalized to 0 to 1 instead
X_train = X_train / 255.0
X_test = X_test / 255.0

print("--- Data Shape ---")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)





Successfully loaded data from local CSV files.
--- Data Shape ---
X_train shape: (60000, 28, 28)
y_train shape: (60000,)
X_test shape: (10000, 28, 28)
y_test shape: (10000,)


Build the Neural Network Model

Used this link to help (https://stackoverflow.com/questions/54282753/how-to-input-2d-numpy-array-into-tensorflow-also-on-how-to-get-matrix-input-an)

In [16]:
model = tf.keras.models.Sequential()
# Because the image is 28x28 the model needs to be set with that shape
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

#Wasnt sure what to set the layers to so i got AI to help decide a good amount
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))

# Because the amount of labels is equal to 10 it needs to be set to that
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='label_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,386 (427.29 KB)

 Trainable params: 109,386 (427.29 KB)

 Non-trainable params: 0 (0.00 B)